<a href="https://colab.research.google.com/github/HackerJacky/Econometrics_project/blob/main/%E5%88%A9%E7%8E%87cpi%E8%88%87m1%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install fredapi

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import yfinance as yf
import datetime as dt
import pandas_datareader.data as web
from fredapi import Fred
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import VECM, coint_johansen
from google.colab import drive
from IPython.display import display
from statsmodels.tsa.api import VAR
from scipy.stats import ttest_rel

In [4]:
fred = Fred(api_key='cb616b95642d785dc1427e4e2f53e61e')
drive.mount('/content/drive')

start = '2015-01-01'
end = '2025-03-01'
SP500=yf.download('^GSPC',start=start,end=end)
SP500_data=SP500['Close']
SP500_data

#聯邦利率
rate = fred.get_series('FEDFUNDS', observation_start=start, observation_end=end).diff().dropna()
#CPI
cpi = fred.get_series('CPIAUCSL', observation_start=start, observation_end=end).diff().dropna()
#M1貨幣流通量
m1 = fred.get_series('M1SL',observation_start=start, observation_end=end).dropna()
m1_yoy = m1.pct_change(12) * 100


Mounted at /content/drive
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
df_all = pd.concat([SP500_data, rate, cpi, m1_yoy], axis=1).dropna()
df_all.columns = ['SP500_ret', 'rate', 'cpi', 'm1_yoy']


In [6]:
for col in df_all.columns:
    result = adfuller(df_all[col])
    print(f"{col}: ADF Statistic = {result[0]:.4f}, p-value = {result[1]:.4f}")

SP500_ret: ADF Statistic = 0.4304, p-value = 0.9826
rate: ADF Statistic = -2.0267, p-value = 0.2750
cpi: ADF Statistic = -5.7028, p-value = 0.0000
m1_yoy: ADF Statistic = -1.9064, p-value = 0.3291


In [7]:
df_diff = df_all.copy()
df_diff['SP500_ret'] = df_diff['SP500_ret'].diff()
df_diff['rate'] = df_diff['rate'].diff()
df_diff['m1_yoy'] = df_diff['m1_yoy'].diff()
df_diff_cleaned = df_diff.replace([float('inf'), float('-inf')], pd.NA).dropna()

In [14]:
model_cleaned = VAR(df_diff_cleaned)

# 重新選擇最適落後期數
lag_selection_cleaned = model_cleaned.select_order(maxlags=12)
selected_lag_cleaned = lag_selection_cleaned.selected_orders['aic']

# 擬合 VAR 模型
fitted_cleaned = model_cleaned.fit(selected_lag_cleaned)

# Granger 因果檢定
granger_results_cleaned = {}
for col in ['rate', 'cpi', 'm1_yoy']:
    granger_results_cleaned[col] = fitted_cleaned.test_causality('SP500_ret', [col], kind='f')

# 衝擊反應函數（IRF）
irf_cleaned = fitted_cleaned.irf(12)

# 預測誤差變異數分解（FEVD）
fevd_cleaned = fitted_cleaned.fevd(12)

# 顯示部分文字摘要與檢定結果
print(fitted_cleaned.summary())


# 印出 Granger 因果檢定結果
for col in ['rate', 'cpi', 'm1_yoy']:
    print(f"\nGranger Causality: {col} → SP500_ret")
    print(granger_results_cleaned[col].summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 03, Jun, 2025
Time:                     09:28:39
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                    18.3469
Nobs:                     60.0000    HQIC:                   14.1815
Log likelihood:          -489.707    FPE:                1.39149e+06
AIC:                      11.5054    Det(Omega_mle):         127755.
--------------------------------------------------------------------
Results for equation SP500_ret
                   coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------
const               164.054821       151.189927            1.085           0.278
L1.SP500_ret         -0.550477         0.326008           -1.689           0.091
L1.rate            1072.416687       503.106555         

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
